In [1]:
pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
from ultralytics import YOLO
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import threading

# Build a YOLO model from pretrained weight with GPU acceleration if available
model = YOLO('yolov10n.pt')

# Flag to control the detection loop
stop_detection = False

# Function to resize image to fit within the canvas while maintaining aspect ratio
def resize_image(image, max_width, max_height):
    width, height = image.size
    ratio = min(max_width / width, max_height / height)
    return image.resize((int(width * ratio), int(height * ratio)), Image.LANCZOS)

# Function to start object detection from webcam
def start_detection_webcam():
    global stop_detection
    stop_detection = False
    cap = cv2.VideoCapture(0)

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while not stop_detection:
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Could not read frame.")
            break

        # Resize the frame to a smaller size before detection
        frame = cv2.resize(frame, (640, 480))

        # Run YOLO inference on the frame
        results = model(frame)
        
        # Extract and draw the results on the frame
        for result in results:
            boxes = result.boxes  # boxes object contains bounding boxes, confidence scores, and class ids
            for box in boxes:
                # Extract bounding box coordinates, confidence score, and class id
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                class_id = int(box.cls[0])
                
                # Draw bounding box on the detection frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # Draw label
                label = f'{model.names[class_id]} {confidence:.2f}'
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Convert frame to PhotoImage and display in the detection Canvas
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = resize_image(img, detection_canvas.winfo_width(), detection_canvas.winfo_height())
        img = ImageTk.PhotoImage(img)
        detection_canvas.create_image(0, 0, anchor=tk.NW, image=img)
        detection_canvas.image = img  # keep a reference to avoid garbage collection
        detection_canvas.update()  # update the canvas to show the new image

    cap.release()
    cv2.destroyAllWindows()

# Function to stop object detection
def stop_detection_func():
    global stop_detection
    stop_detection = True

# Function to start detection in a new thread
def start_detection_thread():
    global stop_detection
    stop_detection = True
    if hasattr(start_detection_thread, 'current_thread') and start_detection_thread.current_thread.is_alive():
        start_detection_thread.current_thread.join()
    stop_detection = False
    start_detection_thread.current_thread = threading.Thread(target=start_detection_webcam)
    start_detection_thread.current_thread.start()

# Create a GUI window
root = tk.Tk()
root.title("YOLOv10 Webcam Detection")

# Configure root window background color
bg_color = '#f0f0f0'  # light gray
root.configure(bg=bg_color)

# Create a Frame for the heading and detection canvas
detection_frame = ttk.Frame(root)
detection_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

# Add Object Detection heading
heading_label = ttk.Label(detection_frame, text="Object Detection", font=("Arial", 16), background=bg_color, foreground='#333333')
heading_label.pack(pady=10)

# Create a Canvas to display the detection results
detection_canvas = tk.Canvas(detection_frame, bg='white')
detection_canvas.pack(fill=tk.BOTH, expand=True)

# Create a Frame for the buttons
button_frame = ttk.Frame(root)
button_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

# Function to create buttons with specified background color
def create_button(text, command, bg_color):
    button = tk.Button(button_frame, text=text, command=command, bg=bg_color, relief=tk.FLAT, font=("Arial", 12), padx=10, pady=5)
    button.pack(pady=10, fill=tk.X)
    return button

# Add buttons for different detection modes
webcam_button = create_button("Detect Webcam", start_detection_thread, '#4CAF50')  # Green
stop_button = create_button("Stop Detection", stop_detection_func, '#f44336')  # Red

# Run the GUI event loop
root.mainloop()

100%|███████████████████████████████████████████████████████████████████████████████████████████| 5.59M/5.59M [00:26<00:00, 218kB/s]
